In [89]:
import networkx as nx
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from qc_grader.graph_util import display_maxcut_widget, QAOA_widget, graphs
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit_optimization import QuadraticProgram
from typing import List
import plotly.graph_objects as go
import matplotlib as mpl
import pandas as pd
from IPython.display import clear_output
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
from qiskit import Aer
from qiskit import QuantumCircuit
from qiskit.visualization import plot_state_city
from qiskit.algorithms.optimizers import COBYLA, SLSQP, ADAM
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from time import time
from copy import copy
from qc_grader.graph_util import display_maxcut_widget, QAOA_widget, graphs
import math
from qiskit.visualization import plot_histogram
import operator
from collections import Counter
from qiskit.circuit.add_control import add_control
import numpy as np



matplotlib.rcParams['figure.dpi'] = 100

### Testing how cirq works

In [92]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator

controls = QuantumRegister(2)
circuit = QuantumCircuit(controls)

cx = Operator([
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0]
])
circuit.unitary(cx, [0, 1], label='cx')

In [93]:
import cirq

# Pick a qubit.
qubit = cirq.GridQubit(0, 0)

# Create a circuit
circuit = cirq.Circuit(
  cirq.X(qubit)**0.5,  # Square root of NOT.
  cirq.measure(qubit, key='m')  # Measurement.
)
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)

Circuit:
(0, 0): ───X^0.5───M('m')───
Results:
m=00000100101011001111


### Binary clustering objective function - classical

In [94]:
def dot(v1, v2):
    return v1[0]*v2[0] + v1[1]*v2[1]

def single_dataset_obj(dataset):
    obj = 0
    
    if (len(dataset)) == 0:
        return 0
    
    for v in dataset:
        obj += dot(v, v)
    
    subtract_term = 0
    for v1 in dataset:
        for v2 in dataset:
            subtract_term += dot(v1, v2)
    subtract_term = subtract_term/len(dataset)
    obj -= subtract_term
    
    return obj

# negative of obj and remove constants, 
# prob of getting the correct cluster should be highest for the cluster with min objective
def prob_clusters(dataset):
    prob = 0
    
    if (len(dataset)) == 0:
        return 0
    
#     for v in dataset:
#         obj += dot(v, v)
    
    subtract_term = 0
    for v1 in dataset:
        for v2 in dataset:
            subtract_term += dot(v1, v2)
    subtract_term = subtract_term/len(dataset)
    prob += subtract_term
    
    return prob

def objective(dataset1, dataset2):
    return single_dataset_obj(dataset1) + single_dataset_obj(dataset2)

def prob(dataset1, dataset2):
    return prob_clusters(dataset1) + prob_clusters(dataset2)

In [95]:
dataset = [(1, 2), (3, 4), (-1, -2), (-3, -4)]

### Let's test if binary clustering algo runs for 2 datapoints

In [96]:
"""Define a custom single-qubit gate."""
class BinaryClusterGateTwoDatapoints(cirq.TwoQubitGate):
    def __init__(self):
        super(BinaryClusterGateTwoDatapoints, self)

    def _num_qubits_(self):
        return 2

    def _unitary_(self):
        return np.array([
           [ 0.29983821,  0.59967641,  0.29983821,  0.67866214],
           [ 0.59967641,  0.        ,  0.59967641, -0.52988339],
           [ 0.29983821,  0.59967641, -0.63658818, -0.38110464],
           [ 0.67866214, -0.52988339, -0.38110464,  0.33674997]
        ])

    def _circuit_diagram_info_(self, args):
        return "U1", "U1"
    
"""Define a custom single-qubit gate."""
class StatePreparationGate(cirq.TwoQubitGate):
    def __init__(self):
        super(StatePreparationGate, self)

    def _num_qubits_(self):
        return 2

    def _unitary_(self):
        return np.array([
           [ 0.29983821,  0.59967641,  0.29983821,  0.67866214],
           [ 0.59967641,  0.        ,  0.59967641, -0.52988339],
           [ 0.29983821,  0.59967641, -0.63658818, -0.38110464],
           [ 0.67866214, -0.52988339, -0.38110464,  0.33674997]
        ])

    def _circuit_diagram_info_(self, args):
        return "U1", "U1"

my_gate = BinaryClusterGateTwoDatapoints()

In [97]:
"""Use the custom two-qubit gate in a circuit."""
circ = cirq.Circuit(
    my_gate.on(*cirq.LineQubit.range(2))
)

print("Circuit with custom two-qubit gate:")
print(circ)

Circuit with custom two-qubit gate:
0: ───U1───
      │
1: ───U1───


In [98]:
qreg = cirq.LineQubit.range(2)
circ = cirq.Circuit(
#     cirq.H(qreg[0]),
#     cirq.H(qreg[1]),
#     cirq.H(qreg[0]),
#     cirq.H(qreg[1]),
    my_gate.on(*qreg)
)

print("Circuit:")
print(circ)

Circuit:
0: ───U1───
      │
1: ───U1───


#### Initial state = [x0.x0, x0.x1, x1.x1, 0] (normalized)

In [120]:
def create_initial_state(point1, point2):
    initial_state = [dot(point1, point1), 
                 dot(point1, point2),
                 dot(point2, point2),
                 0]
    initial_state = initial_state/np.linalg.norm(initial_state)
    return initial_state

In [125]:
initial_state = create_initial_state(dataset[0], dataset[1])

"""Simulate a circuit with a custom gate."""
sim = cirq.Simulator()

res = sim.simulate(circ, initial_state=initial_state)
print(res)

measurements: (no measurements)
output vector: 0.562|00⟩ + 0.648|01⟩ - 0.282|10⟩ - 0.431|11⟩


### Out of probability 00, 01 we see that 01 is higher -> 01 is the most optimal cluster

### Lets see the classical results

In [127]:
dataset1 = np.array([dataset[0]])
dataset2 = np.array([dataset[1]])

In [128]:
print ("Objective for 01 clustering: " , objective(dataset1, dataset2))
print ("Inverse Objective for 01 clustering: ", prob(dataset1, dataset2))

Objective for 01 clustering:  0.0
Inverse Objective for 01 clustering:  30.0


In [129]:
dataset1 = np.array([dataset[0], dataset[1]])
dataset2 = np.array([])

In [130]:
print ("Objective for 00 clustering: " , objective(dataset1, dataset2))
print ("Inverse Objective for 00 clustering: ", prob(dataset1, dataset2))

Objective for 00 clustering:  4.0
Inverse Objective for 00 clustering:  26.0


### Lets test other data points

In [131]:
initial_state = create_initial_state(dataset[0], dataset[2])

"""Simulate a circuit with a custom gate."""
sim = cirq.Simulator()

res = sim.simulate(circ, initial_state=initial_state)
print(res)

measurements: (no measurements)
output vector: 0.692|01⟩ - 0.541|10⟩ + 0.478|11⟩


In [132]:
initial_state = create_initial_state(dataset[0], dataset[3])

"""Simulate a circuit with a custom gate."""
sim = cirq.Simulator()

res = sim.simulate(circ, initial_state=initial_state)
print(res)

measurements: (no measurements)
output vector: 0.086|00⟩ + 0.648|01⟩ - 0.757|10⟩ - 0.011|11⟩


In [133]:
initial_state = create_initial_state(dataset[1], dataset[3])

"""Simulate a circuit with a custom gate."""
sim = cirq.Simulator()

res = sim.simulate(circ, initial_state=initial_state)
print(res)

measurements: (no measurements)
output vector: 0.692|01⟩ - 0.541|10⟩ + 0.478|11⟩


In [134]:
initial_state = create_initial_state(dataset[1], dataset[2])

"""Simulate a circuit with a custom gate."""
sim = cirq.Simulator()

res = sim.simulate(circ, initial_state=initial_state)
print(res)

measurements: (no measurements)
output vector: 0.086|00⟩ + 0.648|01⟩ - 0.082|10⟩ + 0.752|11⟩


In [135]:
initial_state = create_initial_state(dataset[2], dataset[3])

"""Simulate a circuit with a custom gate."""
sim = cirq.Simulator()

res = sim.simulate(circ, initial_state=initial_state)
print(res)

measurements: (no measurements)
output vector: 0.562|00⟩ + 0.648|01⟩ - 0.282|10⟩ - 0.431|11⟩
